In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [7]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=5, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")
opt = Optimizer(stk_object,5,5,3)
clear_output(wait=False)

In [3]:
# Loading targets into stk from file.
stk_object.Target_Loader("../../Input_Files/Targets_15.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [ ]:
stk_object.dt = 30
stk_object.Interpolate = True
res = opt.run(read=True)
opt.cost_function(res[0][0])

In [ ]:
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=30, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

stk_object.dt = 2.5
stk_object.Generate_Holding_Data()

In [ ]:
import matplotlib.pyplot as plt

def compute_orbital_resonance(Individual,error):
    Re = 6378
    J2 = 1082.64e-6
    T_sid = 86164.0905
    mu = 3.986004418e5
    Alt = Individual[0]
    Inc = Individual[1]
    a = Re+Alt
    T = 2*np.pi*(a**3/mu)**0.5
    rate = (-3*J2*(Re/a)**2*np.cos(Inc)-4*np.pi/T_sid*(a**3/mu)**0.5)*np.pi
    i = 1
    while True:
        res = rate*i%(2*np.pi)
        i+=1
        if res < np.radians(error):
            break
    return(T*i/86400,np.radians(res))

incs = np.linspace(60,120,5)
for alt in [580,600,620]:
    plt.plot(incs,[compute_orbital_resonance([alt,inc],5)[0] for inc in incs],label=f"{alt} km")
plt.legend()
# plt.yscale('log')
plt.show()

In [ ]:
stk_object.Plan(slew_rate=1,cone_angle=20.0)
df_1 = stk_object.Planned_Data.copy()
df_1['Slew Rate'] = 1
stk_object.Plan(slew_rate=0,cone_angle=20.0)
df_0 = stk_object.Planned_Data.copy()
df_0['Slew Rate'] = 0
df = pd.concat([df_1, df_0], ignore_index=True, sort=False)

In [ ]:
data_comparison = {}
data_comparison["Unplanned (%)"] = [len(np.unique(stk_object.Holding_Data[stk_object.Holding_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [max(stk_object.Holding_Data[stk_object.Holding_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]
# data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
# data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

In [ ]:
fig = px.scatter(stk_object.Planned_Data.sort_values(by="Target"),x="Time",y="Bin Number",hover_data="Bin Number",color="Satellite",animation_frame="Target")
fig.show()

In [ ]:
df['Satellite'] = df['Satellite'].astype(int)
df['Satellite'] = df['Satellite'].astype(str)
fig = px.scatter(df.sort_values(by="Slew Rate"),x="Cross Track",y="Along Track",
                 color="Satellite",
                 facet_col="Slew Rate",
                 labels={"Cross Track":"Cross Track [deg]",
                         "Along Track":"Along Track [deg]",
                         "Slew Rate":"Slew Rate [deg/s]"})
fig.update_layout(width=800,
                  height=400,
                  title="Planned Data for Various Slewing Rates")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_xaxes(range=(-70,70))
fig.update_yaxes(range=(-70,70))
fig.show()